In [ ]:
import urllib
import urllib2
from scrapy.selector import Selector
from scrapy.http import HtmlResponse
from urlparse import urlsplit, urljoin
# Yeah I know I have scrapy above as well. I like them for different things.
from bs4 import BeautifulSoup
import re
import os

In [ ]:
base_url = 'http://www.cra-arc.gc.ca/formspubs/t1gnrl/llyrs-eng.html'
base_page = urllib2.urlopen(base_url).read()
selection = BeautifulSoup(base_page, 'html.parser')
# print selection.prettify()

### Get a url for each year

In [ ]:
year_items = selection.find(property="mainContentOfPage").ul.find_all('li')
year_pages_dict = {}
year = 2014
for item in year_items:
    year_pages_dict[str(year)] = item.a.get("href")
    year = year - 1
# year_pages_dict

In [ ]:
url_split = urlsplit(base_url)
def completeURL(partial):
    return 'http://' + url_split.netloc + partial

In [ ]:
year_pages_dict_full = {}
for year, url in year_pages_dict.iteritems():
    # Create the url and then append it into the empty array
    url_joined = completeURL(url)
    year_pages_dict_full[year] = url_joined

# Debugging
# year_pages_dict_full

In [ ]:
provincial_links = {}

def provinceUrlListAppend(url_list, flag, year_url, year):
    print year
    if flag == "normal" or flag == "older":
        for url in url_list:
            provincial_links[year].append(completeURL(url))
    elif flag == "reallyOld":
        for url in url_list:
            link = 'http://' + url_split.netloc
            link += urlsplit(year_url).path.rsplit('/', 1)[0]
            link += "/" + url
            provincial_links[year].append(link)

def provinceUrlListProcess(url_list, year_url, year):
    provincial_links[year] = []
    # If it's 'normal' and Alberta is detected, go ahead
    if url_list[0] == 'Alberta' or url_list[0] == ' ':
        url_list = selection.xpath("//main[@property='mainContentOfPage']/ul[1]/li/a/@href").extract()
        provinceUrlListAppend(url_list, "normal", year_url, year)
    else:
        # In some older links, they stuck another unordered list above
        url_list = selection.xpath("//main[@property='mainContentOfPage']/ul[2]/li/a/@href").extract()
        # Another edge case check where they started using relative urls around 1998
        if url_list[0][0] == "/":
            provinceUrlListAppend(url_list, "older", year_url, year)
        else:
            provinceUrlListAppend(url_list, "reallyOld", year_url, year)

for year, year_url in year_pages_dict_full.iteritems():
    year_page = urllib2.urlopen(year_url).read()
    selection = Selector(text=year_page)
    provt_pages = selection.xpath("//main[@property='mainContentOfPage']/ul/li/a/text()").extract()
    provinceUrlListProcess(provt_pages, year_url, year)

# print provincial_links

In [ ]:
def downloadPdfProv(prov_url):
    print prov_url
    # Go to the provincial page, find the doc that has the tax brackets in it
    page = urllib2.urlopen(prov_url).read()
    page_text = BeautifulSoup(page, 'html.parser')
    try: 
        download_url = completeURL(page_text.find('a', text = re.compile('428')).get("href"))

        # Go to the download page and find the pdf url
        page = urllib2.urlopen(download_url).read()
        page_text = BeautifulSoup(page, 'html.parser')
        pdf_url = completeURL(page_text.find('a', text = re.compile('pdf')).get("href"))
        pdf_url

        # Save the PDF locally
        cwd = os.getcwd()
        download_folder = os.path.join(cwd, "IncomeTaxForms")
        file_name = os.path.join(download_folder, pdf_url.split("/")[-1])

        # Speeds things up if the file already exists
        if not os.path.isfile(file_name):
            urllib.urlretrieve(pdf_url, file_name)
    except:
        print "FAILED"

# Works for all provinces 1999 - 2014, except Quebec

In [ ]:
year = "1999"
year_links = provincial_links[year]
for prov_url in year_links:
    downloadPdfProv(prov_url)

# Download federal 1999-2014

In [ ]:
def downloadPdfFed(year_url):
    print year_url
    # Go to the provincial page, find the doc that has the tax brackets in it
    page = urllib2.urlopen(year_url).read()
    page_text = BeautifulSoup(page, 'html.parser')
    try: 
        download_url = completeURL(page_text.find('a', text = re.compile('Schedule 1')).get("href"))

        # Go to the download page and find the pdf url
        page = urllib2.urlopen(download_url).read()
        page_text = BeautifulSoup(page, 'html.parser')
        pdf_url = completeURL(page_text.find('a', text = re.compile('pdf')).get("href"))
        pdf_url

        # Save the PDF locally
        cwd = os.getcwd()
        download_folder = os.path.join(cwd, "IncomeTaxForms")
        file_name = os.path.join(download_folder, pdf_url.split("/")[-1])

        # Speeds things up if the file already exists
        if not os.path.isfile(file_name):
            urllib.urlretrieve(pdf_url, file_name)
    except:
        print "FAILED"

In [ ]:
years = xrange(1999, 2015)
for year in years:
    year_url = provincial_links[str(year)][0]
    downloadPdfFed(year_url)